In [2]:
import tensorflow as tf

**tf.keras的api**

首先，tf.keras與我們所熟知的keras是不太一樣的，以前我們所使用的keras，他的backend其實也是用tensorflow，且Keras有自己做好的模型wrapper。

而現在TF2.0 大力將keras整進去，一般來說，在tf.keras下，比較常使用的像是 tf.keras.layer , tf.keras.losses,tf.keras.mertrics,tf.keras.optimzer 等等

**tf.keras.metrics:**

主要像是用在training model的過程，常會用以紀錄像是loss或者accuracy等等的模型數值。



假如要建立一些模型衡量指標:

In [ ]:
Model_acc = tf.keras.metrics.Accurcy()
#or
Model_mean = tf.keras.metrics.Mean()

當我們要update或者說增加裡面的數值:

In [ ]:
Model_acc.update_state(y,pred)
#or
Model_mean.update_state(current_loss)

在每一次迭代(iteration)或每一回合(epoch)，在模型裡面print出這些數值:



In [ ]:
#training steps
print(step,'Train_loss:',Model_mean.result().numpy(),
      'Train_Acc',Model_acc.result().numpy())

當每個epoch跑完要清楚裡面的數值時:

In [ ]:
Model_mean.reset_states()
Model_acc.reset_states()

tf.keras也有提供資料前處理的API:tf.keras.preoprocessing，包含特徵工程以及資料增強等等的前處理方法，可應用於文字辨識、影音辨識等多種情境

以下是一個常見的資料增強(data Augmentation)的方法:Random Shift

In [ ]:
(x,y),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
img_shifted = tf.keras.preoprocessing.image.random_shift(
    x,
    wrg = 0.2,
    hrg = 0.2,
    fill_mode = 'constant'
)

**2.Model Define：**

在Model定義裡像是我們會使用tf.keras.Sequential，它是一個序列模型，可以讓我們輕易地透過這個API來堆疊網路並建立序列化模型。

在各層的定義就會使用tf.keras.layers來定義每一層 (Ex: neuron數、activation function等等)。

而若是想看一下參數的化，model.trainable_variables可以拿取model中要train的variable (Ex: weight , bias等)。

最後就丟進去compile即可！

*補充:*

*tf.keras還有一個定義模型的方式:* **Functional API**

*序列化模型的一個缺點是每一層僅能做到單輸入單輸出，無法多輸入多輸出；想要建立一個較為複雜的模型的話，就要使用Functional API*



In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(64,activation=tf.nn.relu),
    tf.keras.layers.Dense(32,activation=tf.nn.relu),
    tf.keras.layers.Dense(16,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.relu)
])

model.build(input_shpae=[-1,28*28])

想要custom layer:

在class中主要就是要實現init、call跟build的方法，然後記得要繼承tf.keras.layer。

​ init : 主要就是初始化，以及繼承

​ call:執行向前傳導

​ build: 輸入shape，定義viarble的等等

In [3]:
#定義類別
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

#呼叫類別中的物件/方法  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Instructions for updating:
Please use `layer.add_weight` method instead.
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.18886581, -0.24676901,  0.5485528 ,  0.14527285,  0.35803646,
         0.4273072 ,  0.39134473,  0.48106366,  0.35301638,  0.2746222 ],
       [ 0.03609514, -0.47452742,  0.6233738 ,  0.5700552 , -0.29246294,
        -0.62321186, -0.5627171 , -0.43030277, -0.08164567, -0.61876553],
       [ 0.11935896, -0.11348706, -0.57339066,  0.12509286, -0.4366442 ,
         0.45993334, -0.28063262, -0.20672333,  0.38035625, -0.38883123],
       [ 0.10426813,  0.08950424, -0

而自定義一個網路架構也是非常類似的，但他所繼承的類別就是tf.keras.Model

In [4]:
#定義類別
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters
 
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()
 
    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()
 
    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()
 
  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)
 
    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)
 
    x = self.conv2c(x)
    x = self.bn2c(x, training=training)
 
    x += input_tensor
    return tf.nn.relu(x)
 
#呼叫類別中的物件/方法   
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/batch_normalization/moving_mean:0', 'resnet_identity_block/batch_normalization/moving_variance:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/batch_normalization_1/moving_mean:0', 'resnet_identity_block/batch_normalization_1/moving_variance:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0', 'resnet_identity_block/batch_normalization_2

3.Model training

一般來說，Model training的api會分為四類：

model.compile

model.fit

model.evaluate

model.predict

這四項。其實，這些api也很直觀，就是整個ML training的pipeline。

若不用tf.keras的api，就像之前Lab一樣，要自己寫tf.GradientTape()來訓練參數。

In [ ]:
model.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['mean_squared_error'])

history = model.fit(X_train.values ,y_train.values, epochs=100, validation_split = 0.1)

model.evaluate(X_val)

在訓練過程中，我們經常使用Callback函數來監控模型狀況；其中又屬Earlystopping最常使用

一般來說會使用驗證資料集的loss和acc來做Early Stopping，主要參數為:

min_delta: 改善的幅度門檻

patience: 在多少個epoch內若驗證集的改善程度未達設定之門檻即停止訓練

mode: 當訓練狀況停止下降為'min'；反之為'max'；自動選擇則是'auto'



In [ ]:
tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss', min_delta = 0, patience = 0, mode = 'auto')

4.Save / Load Model:

一般來說，跑完model，我們會希望能把model 儲存下來，或者說跑到一半中斷想要save model，之後可以直接Load model繼續train。

儲存的方式有兩種 save/load weight 、save /load model 。

Save weight的方式是比較輕量級的方法，但是模型架構等等都要先定義好且相同。

Save model就是直接全部存下來，不需要先定義模型架構。

In [ ]:
#Save weight
model.save_weights('weights.ckpt')

#Load weight
model = create_model()
model.load_weights('weights.ckpt')

# save model
model.save('model.h5')

# load model
model = tf.keras.models.load_model('model.h5')

另外，如果我們在訓練過程中想直接儲存權重的話，也可以直接使用ModelCheckpoint來儲存模型每一個epoch的權重

使用該函數的好處在於這個API可將模型使用的權重紀錄下來，當模型因為訓練過久或是系統不穩定而中斷訓練時，可透過記錄繼續訓練

In [ ]:
#直接儲存最佳權重之模型
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',
                             verbose = 1, save_best_only = True, 
                             mode = 'max')